In [1]:
import pandas as pd

df = pd.read_csv("ai_102.csv")

In [2]:
df.head(1)

,Unnamed: 0,module,unit,topic_name,content
0,0,Develop generative AI apps in Azure - Training,Plan and prepare to develop AI solutions on Azure,Introduction,# Introduction\n\nCompleted 100 XP\n\n * 1 mi...


In [52]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
import os

azure_search_key = os.getenv('AZURE_SEARCH_KEY')
azure_search_endpoint = os.getenv('AZURE_SEARCH_ENDPOINT')

credential = AzureKeyCredential(azure_search_key)

search_index_client = SearchClient(endpoint=azure_search_endpoint, 
                                   index_name="exam-simulator-idx", 
                                   credential=credential)


In [2]:
from langchain_openai import AzureOpenAIEmbeddings

azure_embeddings_deployment = os.getenv('AZURE_EMBEDDINGS_DEPLOYMENT')
azure_oai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
azure_oai_key = os.getenv('AZURE_OPENAI_API_KEY')

embeddings = AzureOpenAIEmbeddings(azure_deployment=azure_embeddings_deployment,
                                       azure_endpoint=azure_oai_endpoint,
                                       api_key=azure_oai_key,
                                       api_version="2023-05-15")

In [6]:
file_path = "../data/ai-102 data.json"

In [7]:
import json

with open(file_path, "r", encoding='utf-8') as file:
        data = json.load(file)
#data_json = json.loads(data)

In [26]:

resultado = next((item for item in data if item.get('videos') is not None), None)

print(resultado)


{'course': 'Course AI-102T00-A: Develop AI solutions in Azure - Training', 'module': 'Develop natural language solutions in Azure - Training', 'unit': 'Build a conversational language understanding model', 'topic': 'Introduction', 'text': "Natural language processing (NLP) is a common AI problem in which software must be able to work with text or speech in the natural language form that a human user would write or speak. Within the broader area of NLP, natural language understanding (NLU) deals with the problem of determining semantic meaning from natural language - usually by using a trained language model. A common design pattern for a natural language understanding solution looks like this: In this design pattern: An app accepts natural language input from a user. A language model is used to determine semantic meaning (the user's intent ). The app performs an appropriate action. Azure AI Language enables developers to build apps based on language models that can be trained with a re

In [51]:
data

[{'course': 'Course AI-102T00-A: Develop AI solutions in Azure - Training',
  'module': 'Develop generative AI apps in Azure - Training',
  'unit': 'Plan and prepare to develop AI solutions on Azure',
  'topic': 'Introduction',
  'text': "The growth in the use of artificial intelligence (AI) in general, and generative AI in particular means that developers are increasingly required to create comprehensive AI solutions. These solutions need to combine machine learning models, AI services, prompt engineering solutions, and custom code. Microsoft Azure provides multiple services that you can use to create AI solutions. However, before embarking on an AI application development project, it's useful to consider the available options for services, tools, and frameworks as well as some principles and practices that can help you succeed. This module explores some of the key considerations for planning an AI development project, and introduces Azure AI Foundry ; a comprehensive platform for AI 

In [57]:
docs = []

#for index, row in df.iterrows():
i = 0
for item in data:
    i += 1
    docs.append(
        {
            "docid": str(i),
            "exam": item['course'],
            "module": item['module'],
            "unit": item['unit'],
            "topic": item['topic'],
            "content": item['text'],
            "links": item['links'] if item['links'] is not None else [],
            "images": item['images'] if item['images'] is not None else [],
            "videos": item['videos'] if item['videos'] is not None else [],
            "exercise_url": item['exercise'],
            "content_vector": embeddings.embed_query(item['text'])
        }
    )

In [59]:
result = search_index_client.upload_documents(documents=docs)

In [60]:
search_index_client.get_document_count()

204